# LSTM model to a Sequence Array. Improvements Of Compression Rate

##

In [364]:
from Bio import SeqIO
import numpy as np
import copy
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
#import matplotlib.pyplot as plt

# NN NN Architectrue
from keras.models import Model#, Sequential
from keras.layers import Input, LSTM, Flatten, Dense, Permute, Dropout
from keras.optimizers import Adam
from keras.regularizers import L1


In [320]:
# Pre-Processing Functions
def seq2bool(seq, amines):
    return (np.expand_dims(np.array(list(seq)),1) == np.expand_dims(amines,1).T)

def qual2bool(qual):
    uq = np.unique(qual)
    return (np.expand_dims(qual,1) == np.expand_dims(uq,1).T)

def qual2R(qual):
    qual = np.array(qual)/(40.0)
    m,s = np.mean(qual), np.std(qual)
    qual = (qual - m)/s
    return qual, m, s;

def R2qual(qual,m,s):
    qual = qual *s + m
    qual = qual*40.0;
    return qual;

#  test
q1 = np.array([1,2,3,0])
q2, m, s = qual2R(q1)
q3 = R2qual(q2,m,s);
sum((q1 - q3)**2)

0.0

In [321]:
# Input- Output Construction Functions
def mk_xt(sequence_array,qual_array, N = 10**2, method = 's'):
    if method == 'q':
        # Quality
        x1 = sequence_array[:N,:];
        x1 = np.moveaxis(np.expand_dims(x1,2), 1,2)
        
        x2 = qual_array[:N,:];
        x2 = np.moveaxis(np.expand_dims(x2,2), 1,2)        

        x = np.concatenate([x1,x2], axis = 2)
        t = np.roll(x2,1)

    elif method == 's':    
        # Sequence
        x = np.moveaxis(np.expand_dims(sequence_array[:N,:],2), 1,2)
        
        t = np.roll(x,1)

    elif method == 'q+s':
        # Sequence + Quality
        x1 = sequence_array[:N,:];
        x1 = np.moveaxis(np.expand_dims(x1,2), 1,2)
        
        x2 = qual_array[:N,:];
        x2 = np.moveaxis(np.expand_dims(x2,2), 1,2)        
        
        x = np.concatenate([x1,x2], axis = 2)
        t = np.roll(x2,1)
    return x,t

def xt2vec(x,t,timesteps,regressors):
    # Vectorizations (one-hot encoding likewise)
    X = [];
    for i in range(timesteps):
        X.append(np.roll(x,-i))

    X = np.concatenate(X, axis = 1)
    t = t[:,0:1,0:regressors];

    t = np.moveaxis(t, 2, 1);
    return X, t;

def mk_arh(units, timesteps, predictors, regressors,learning_rate):
    # Make Architecture
    i = Input(shape=(timesteps,predictors),name = 'Input')
    l = LSTM (units, return_sequences = True, use_bias = False,name = 'LSTM') (i)
    l *= (1.0)/np.tanh(1).astype(float);
    l = Flatten(name = 'Flatten')(l)
    #l = Dropout(0.95,name = 'Dropout')(l)
    l = Dense(regressors, name = 'Dense', activation = 'linear', use_bias = False) (l)

    regressor = Model (inputs = i, outputs = l)
    regressor.compile(optimizer = Adam(lr=learning_rate), loss = 'mean_squared_error', metrics=['mse'])
    return regressor;

def n_params(nn):
    n = 0;
    w = nn.get_weights()
    for ww in w:
        n += ww.size
    return n;

In [322]:
!head -n 12 ../data/E_4_20_1_short_example.fq

'head' is not recognized as an internal or external command,
operable program or batch file.


In [323]:
qual_array, sequence_array = [],[]
nucleotids = ['A', 'C', 'G', 'T']
with open("../data/E_4_20_1_short_example.fq") as input_handle:
    for read in SeqIO.parse(input_handle, "fastq"):
        sequence_array.append(seq2bool(read.seq, nucleotids))
        qual_array.extend(read._per_letter_annotations["phred_quality"])

# Concatenate sequence
sequence_array = 1*np.concatenate(sequence_array, axis = 0)
qual_array,m,s = qual2R(np.array([qual_array]).T)

In [324]:
N = 2**12
method = 's'; #'s', 'q+s'
n_timesteps = 4;
n_units = 1;
n_regressors = 4;
epochs = 3*10**2;
learning_rate = 0.075;

x,t = mk_xt(sequence_array,qual_array, N, method); #
X,t = xt2vec(x,t,n_timesteps,n_regressors);
n_predictors = X.shape[2];

nn = mk_arh(n_units, n_timesteps,n_predictors, n_regressors, learning_rate)
nn.fit(X, t, epochs = epochs, verbose = 0) # approximate
n = n_params(nn)

C:\anaconda\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



In [325]:
y = nn(X)
y = np.expand_dims(y, axis = 2);
e = y - t

In [326]:
# Error Decay
regressor_df = pd.DataFrame.from_dict(
    {
        'loss': nn.history.history['loss'],
    }
)

error_df = pd.DataFrame.from_dict(
    {
        'A_error': np.array(e[:,0,0]),
        'C_error': np.array(e[:,1,0]),
        'G_error': np.array(e[:,2,0]),
        'T_error': np.array(e[:,3,0])
    }
)

fig = px.line(regressor_df, y="loss")
fig.show()

fig = px.area(error_df, y='A_error')
fig.show()

fig = px.area(error_df, y='C_error')
fig.show()

fig = px.area(error_df, y='G_error')
fig.show()

fig = px.area(error_df, y='T_error')
fig.show()

In [327]:
# Error Decay
regressor_df = pd.DataFrame.from_dict(
    {
        'loss': nn.history.history['loss'],
    }
)

error_df = pd.DataFrame.from_dict(
    {
        'error': np.array(e[:,0,0])
    }
)


fig = px.area(error_df, y='error')
fig.show()

In [328]:
# Summary
nn.summary()
print(nn.input_shape)
print(nn.output_shape)
print(X.shape)
print(t.shape)
print(n_params(nn))

Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 4, 4)]            0         
                                                                 
 LSTM (LSTM)                 (None, 4, 1)              20        
                                                                 
 tf.math.multiply_17 (TFOpLa  (None, 4, 1)             0         
 mbda)                                                           
                                                                 
 Flatten (Flatten)           (None, 4)                 0         
                                                                 
 Dense (Dense)               (None, 4)                 16        
                                                                 
Total params: 36
Trainable params: 36
Non-trainable params: 0
______________________________________________________________

In [329]:
(abs(e).round()>0).sum()

1159

In [330]:
# Error terms
print((abs(e).round()>0).mean())
M = ((abs(e).round()>0).sum() + n + n_timesteps);
print('Compression rate')
print( float(N) / float(M))

0.07073974609375
Compression rate
3.416180150125104


In [331]:
N

4096

In [332]:
M

1199

In [333]:
(abs(e).round()>0).sum()

1159

In [334]:
nn.get_weights()

[array([[-1.2787451e+00, -7.1286397e+00,  6.8513303e+00, -1.7250370e+01],
        [ 1.2710035e+01, -2.2203376e+00,  6.6305227e+00, -2.5899458e+00],
        [-4.4051638e-01, -1.6591515e+01,  2.5830373e-01,  1.0508599e-02],
        [-1.1929075e+00, -7.1266246e+00,  4.6302480e-01, -3.4060776e-03]],
       dtype=float32),
 array([[-1.6294136 , 48.09998   ,  0.16715296, -4.034516  ]],
       dtype=float32),
 array([[ 2.1779156, -9.042852 ,  3.0826492,  5.1808095],
        [ 1.0439515,  5.4312906, -9.9794445,  3.8699577],
        [ 1.501137 ,  5.1820703,  6.2386017, -9.939441 ],
        [ 0.4941159,  4.485215 ,  5.304312 ,  5.4793367]], dtype=float32)]

In [335]:
(error_df.abs().round()>0).sum()

error    1159
dtype: int64

In [336]:
s = 1.0*(error_df.abs().round()>0);
s = np.expand_dims(s,1);  print(sum(s))

[[1159.]]


In [337]:
error_int = pd.DataFrame.from_dict(
    {
        'error': np.where(s)[0]
    }
)


fig = px.area(error_int.diff(), y='error')
fig.show()

In [355]:
x_int = error_int.diff().fillna(0);
scale = x_int.max().values;
x_int = x_int/scale
x_int = np.expand_dims(x_int,1)
t_int = np.roll(x_int,-1)
print(x_int.shape)
print(t_int.shape)

(1159, 1, 1)
(1159, 1, 1)


In [356]:
n_timesteps = 4;
n_units = 2;
n_regressors = 1;
epochs = 2*10**2;
learning_rate = 0.075;

X2,z = xt2vec(x_int,t_int,n_timesteps,n_regressors);
T = t_int;
print(T.shape)
print(X2.shape)

n_predictors = X2.shape[2];
nn = mk_arh(n_units, n_timesteps,n_predictors, n_regressors, learning_rate)
nn.fit(X2, T, epochs = epochs, verbose = 1) # approximate
n = n_params(nn)

n2, n2_timesteps = copy.deepcopy(n),copy.deepcopy(n_timesteps);

(1159, 1, 1)
(1159, 4, 1)
Epoch 1/200


C:\anaconda\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



37/37 [==============================] - 2s 2ms/step - loss: 0.0205 - mse: 0.0205
Epoch 2/200
37/37 [==============================] - 0s 3ms/step - loss: 0.0044 - mse: 0.0044
Epoch 3/200
37/37 [==============================] - 0s 3ms/step - loss: 4.1890e-04 - mse: 4.1890e-04
Epoch 4/200
37/37 [==============================] - 0s 2ms/step - loss: 1.6829e-04 - mse: 1.6829e-04
Epoch 5/200
37/37 [==============================] - 0s 2ms/step - loss: 1.0374e-04 - mse: 1.0374e-04
Epoch 6/200
37/37 [==============================] - 0s 2ms/step - loss: 1.5188e-04 - mse: 1.5188e-04
Epoch 7/200
37/37 [==============================] - 0s 3ms/step - loss: 1.2729e-04 - mse: 1.2729e-04
Epoch 8/200
37/37 [==============================] - 0s 3ms/step - loss: 6.7926e-05 - mse: 6.7926e-05
Epoch 9/200
37/37 [==============================] - 0s 3ms/step - loss: 5.6374e-05 - mse: 5.6374e-05
Epoch 10/200
37/37 [==============================] - 0s 2ms/step - loss: 6.0002e-05 - mse: 6.0002e-05
Epoch 1

37/37 [==============================] - 0s 2ms/step - loss: 2.5237e-05 - mse: 2.5237e-05
Epoch 81/200
37/37 [==============================] - 0s 3ms/step - loss: 2.2967e-04 - mse: 2.2967e-04
Epoch 82/200
37/37 [==============================] - 0s 3ms/step - loss: 8.3693e-05 - mse: 8.3693e-05
Epoch 83/200
37/37 [==============================] - 0s 3ms/step - loss: 6.7938e-05 - mse: 6.7938e-05
Epoch 84/200
37/37 [==============================] - 0s 3ms/step - loss: 8.3011e-05 - mse: 8.3011e-05
Epoch 85/200
37/37 [==============================] - 0s 2ms/step - loss: 4.8041e-05 - mse: 4.8041e-05
Epoch 86/200
37/37 [==============================] - 0s 2ms/step - loss: 1.8843e-05 - mse: 1.8843e-05
Epoch 87/200
37/37 [==============================] - 0s 3ms/step - loss: 1.2557e-05 - mse: 1.2557e-05
Epoch 88/200
37/37 [==============================] - 0s 3ms/step - loss: 7.4732e-06 - mse: 7.4732e-06
Epoch 89/200
37/37 [==============================] - 0s 3ms/step - loss: 1.0261e-05 -

37/37 [==============================] - 0s 2ms/step - loss: 1.4431e-05 - mse: 1.4431e-05
Epoch 160/200
37/37 [==============================] - 0s 2ms/step - loss: 1.6805e-05 - mse: 1.6805e-05
Epoch 161/200
37/37 [==============================] - 0s 2ms/step - loss: 1.7254e-05 - mse: 1.7254e-05
Epoch 162/200
37/37 [==============================] - 0s 2ms/step - loss: 5.1519e-05 - mse: 5.1519e-05
Epoch 163/200
37/37 [==============================] - 0s 2ms/step - loss: 2.5662e-05 - mse: 2.5662e-05
Epoch 164/200
37/37 [==============================] - 0s 2ms/step - loss: 1.4507e-05 - mse: 1.4507e-05
Epoch 165/200
37/37 [==============================] - 0s 2ms/step - loss: 1.1955e-05 - mse: 1.1955e-05
Epoch 166/200
37/37 [==============================] - 0s 2ms/step - loss: 2.7224e-05 - mse: 2.7224e-05
Epoch 167/200
37/37 [==============================] - 0s 2ms/step - loss: 1.9703e-05 - mse: 1.9703e-05
Epoch 168/200
37/37 [==============================] - 0s 2ms/step - loss: 1.0

In [357]:
nn.summary()

Model: "model_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 4, 1)]            0         
                                                                 
 LSTM (LSTM)                 (None, 4, 2)              24        
                                                                 
 tf.math.multiply_20 (TFOpLa  (None, 4, 2)             0         
 mbda)                                                           
                                                                 
 Flatten (Flatten)           (None, 8)                 0         
                                                                 
 Dense (Dense)               (None, 1)                 8         
                                                                 
Total params: 32
Trainable params: 32
Non-trainable params: 0
______________________________________________________________

In [363]:
y2 = nn(X2)
y2 = np.expand_dims(y2, axis = 2);
e2 = y2 - T
e2 = e2 * scale;

In [359]:

# Error Decay
regressor2_df = pd.DataFrame.from_dict(
    {
        'loss': nn.history.history['loss'],
    }
)

error2_df = pd.DataFrame.from_dict(
    {
        'A_error': np.array(e2[:,0,0]),
    }
)

fig = px.line(regressor2_df, y="loss")
fig.show()

fig = px.scatter(error2_df.abs().round(), y='A_error')
fig.show()

In [360]:
error2_df.abs().round().sum()

A_error    1.0
dtype: float64

In [361]:
## Error terms
print((abs(e2).round()>0).mean())
M = ((abs(e2).round()>0).sum() + n + n_timesteps + n2 + n2_timesteps);
print('Compression rate')
print( float(N) / float(M))

0.0008628127696289905
Compression rate
56.10958904109589
